In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('Reviews.csv')

In [3]:
import string

def text_process(mess):

    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    punctuation = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    punctuation = ''.join(punctuation)
    
    # Lowering characters
    punctuation = punctuation.lower()
    
    # Now just remove any stopwords
    return ' '.join([word for word in punctuation.split() if word.lower() not in STOPWORDS])

# Stemming function

def stemmer (msg):
    msg = msg.split()
    words = ""
    for i in msg:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [4]:
#assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment
# remove score = 3

df['sentiment'] = df['Score'].apply(lambda rating : +1 if rating > 3 else -1 if rating < 3 else 0)

In [5]:
df['sentimentt'] = df['sentiment'].replace({-1 : 'negative'})
df['sentimentt'] = df['sentimentt'].replace({1 : 'positive'})
df['sentimentt'] = df['sentimentt'].replace({0 : 'neutral'})

#Some data analysis
# fig = px.histogram(df, x="sentimentt")
# fig.update_traces(marker_color="indianred",marker_line_color='rgb(8,48,107)',
#                  marker_line_width=1.5)
# fig.update_layout(title_text='Product Sentiment')
# fig.show()

In [6]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final
df['Text'] = df['Text'].apply(remove_punctuation)
df = df.dropna(subset=['Summary'])
df['Summary'] = df['Summary'].apply(remove_punctuation)
#Applying your function to 2 differents variable containing our messages.


In [7]:
# dfNew = df[['Summary','sentiment']]

In [8]:
# random split train and test data
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.8]
test = df[df['random_number'] > 0.8]

In [9]:
# count vectorizer:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Summary'])
test_matrix = vectorizer.transform(test['Summary'])

In [35]:
# Logistic Regression
lr = LogisticRegression(solver='liblinear', max_iter=400)

In [36]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

In [37]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
predictions = lr.predict(X_test)

In [39]:
# find accuracy, precision, recall:
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[11053,  2030,  2051],
       [  542,  1920,   803],
       [ 5700,  5203, 91170]], dtype=int64)

In [40]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

          -1       0.64      0.73      0.68     15134
           0       0.21      0.59      0.31      3265
           1       0.97      0.89      0.93    102073

    accuracy                           0.86    120472
   macro avg       0.61      0.74      0.64    120472
weighted avg       0.91      0.86      0.88    120472



In [41]:
new_reviews = []
new_reviews.append('i love flowers')
X_new = vectorizer.transform(new_reviews)
print(lr.predict(X_new)[0])


1
